In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, GRU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler

np.random.seed(0) 
tf.random.set_seed(0)

### Loading data and preprocessing

In [2]:
df = pd.read_csv('Monthly_oil_data.csv', index_col='Production Date', parse_dates=True)
df['LogVolume'] = np.log(df['Volume'])
df['DiffLogVolume'] = df['LogVolume'].diff()
df.head()

,Volume,LogVolume,DiffLogVolume
Production Date,,,
2015-01-01,67515553.0,18.027869,NaN
2015-02-01,61664960.0,17.937226,-0.090642
2015-03-01,66342950.0,18.010348,0.073122
2015-04-01,67719040.0,18.030878,0.020530
2015-05-01,66484591.0,18.012481,-0.018397


### Supervised Dataset

In [3]:
series = df['DiffLogVolume'].to_numpy()[1:] 

T = 12
X = [] 
Y = []  
for t in range(len(series) - T):
    x = series[t:t+T] 
    X.append(x) 
    y = series[t+T] 
    Y.append(y)

X = np.array(X).reshape(-1, T) 
Y = np.array(Y)
N = len(X) 
print('X.shape', X.shape, 'Y.shape', Y.shape)

X.shape (112, 12) Y.shape (112,)


### Train Test Split

In [4]:
Ntest = 12  
train = df.iloc[:-Ntest] 
test = df.iloc[-Ntest:] 

Xtrain, Ytrain = X[:-Ntest], Y[:-Ntest] 
Xtest, Ytest = X[-Ntest:], Y[-Ntest:]

### Scaling

In [5]:
scaler_x = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

Xtrain = scaler_x.fit_transform(Xtrain)
Xtrain = Xtrain.reshape(Xtrain.shape[0], Xtrain.shape[1], 1)

Xtest = scaler_x.transform(Xtest)
Xtest = Xtest.reshape(Xtest.shape[0], Xtest.shape[1], 1)

Ytrain = scaler_y.fit_transform(Ytrain.reshape(-1, 1))
Ytest = scaler_y.transform(Ytest.reshape(-1, 1))

### Indexing

In [6]:
train_idx = df.index <= train.index[-1] 
test_idx = ~train_idx 

### Manual Grid Search

In [7]:
def build_and_train_lstm(units, dropout_rate, learning_rate, epochs, batch_size):
    model = Sequential([
        Input(shape =(T, 1)),
        LSTM(units, activation='tanh', return_sequences=True),
        Dropout(dropout_rate),
        LSTM(units, activation='tanh'),
        Dropout(dropout_rate),
        Dense(1)
    ])
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=15,  # Stop if no improvement for 15 epochs
        restore_best_weights=True,  # Restore weights from best epoch
        verbose=0
    )
    
    history = model.fit(
        Xtrain, Ytrain,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(Xtest, Ytest),
        callbacks=[early_stop],
        verbose=0,
        shuffle=False
    )
    
    # Get final validation loss
    val_loss = history.history['val_loss'][-1]
    best_val_loss = min(history.history['val_loss'])
    
    return model, val_loss, history

In [8]:
# Define hyperparameter grid
hyperparameters = {
    'units': [32, 50, 64, 128],
    'dropout_rate': [0.1, 0.2, 0.3],
    'learning_rate': [0.001, 0.0001],
    'epochs': [50, 100],
    'batch_size': [16, 32]
}

# Manual grid search
results = []

for units in hyperparameters['units']:
    for dropout in hyperparameters['dropout_rate']:
        for lr in hyperparameters['learning_rate']:
            for epochs in hyperparameters['epochs']:
                for batch_size in hyperparameters['batch_size']:

                    K.clear_session() 
                    
                    print(f"Testing: units={units}, dropout={dropout}, lr={lr}, "
                          f"epochs={epochs}, batch_size={batch_size}")
                    
                    model, val_loss, history = build_and_train_lstm(
                        units, dropout, lr, epochs, batch_size
                    )
                    
                    results.append({
                        'units': units,
                        'dropout': dropout,
                        'learning_rate': lr,
                        'epochs': epochs,
                        'batch_size': batch_size,
                        'val_loss': val_loss
                    })
                    
                    print(f"  Validation Loss: {val_loss:.6f}\n")

# Convert to DataFrame and sort
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('val_loss')

print("\nTop 5 Configurations:")
print(results_df.head())

# Use best hyperparameters
best_params = results_df.iloc[0]
print(f"\nBest Hyperparameters:")
print(best_params)


Testing: units=32, dropout=0.1, lr=0.001, epochs=50, batch_size=16
  Validation Loss: 0.024159

Testing: units=32, dropout=0.1, lr=0.001, epochs=50, batch_size=32
  Validation Loss: 0.024429

Testing: units=32, dropout=0.1, lr=0.001, epochs=100, batch_size=16
  Validation Loss: 0.024485

Testing: units=32, dropout=0.1, lr=0.001, epochs=100, batch_size=32
  Validation Loss: 0.024565

Testing: units=32, dropout=0.1, lr=0.0001, epochs=50, batch_size=16
  Validation Loss: 0.024437

Testing: units=32, dropout=0.1, lr=0.0001, epochs=50, batch_size=32
  Validation Loss: 0.024104

Testing: units=32, dropout=0.1, lr=0.0001, epochs=100, batch_size=16
  Validation Loss: 0.024494

Testing: units=32, dropout=0.1, lr=0.0001, epochs=100, batch_size=32
  Validation Loss: 0.024175

Testing: units=32, dropout=0.2, lr=0.001, epochs=50, batch_size=16
  Validation Loss: 0.023757

Testing: units=32, dropout=0.2, lr=0.001, epochs=50, batch_size=32
  Validation Loss: 0.024781

Testing: units=32, dropout=0.2,

In [9]:
def build_and_train_lstm_2(units, dropout_rate, learning_rate, batch_size, max_epochs=200):
    """
    Build and train LSTM with early stopping
    """
    # Build model
    model = Sequential([
        Input(shape=(T, 1)),
        LSTM(units, activation='tanh', return_sequences=True),
        Dropout(dropout_rate),
        LSTM(units, activation='tanh'),
        Dropout(dropout_rate),
        Dense(1)
    ])
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    
    # Define early stopping callback
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=15,  # Stop if no improvement for 15 epochs
        restore_best_weights=True,  # Restore weights from best epoch
        verbose=0
    )
    
    # Train with early stopping
    history = model.fit(
        Xtrain, Ytrain,
        epochs=max_epochs,  # Set high, early stopping will stop earlier
        batch_size=batch_size,
        validation_data=(Xtest, Ytest),
        callbacks=[early_stop],
        verbose=0,
        shuffle=False
    )
    
    # Get best validation loss and actual epochs trained
    val_loss = min(history.history['val_loss'])
    epochs_trained = len(history.history['loss'])
    
    return model, val_loss, epochs_trained, history

# Define hyperparameter grid
hyperparameters = {
    'units': [32, 50, 64],
    'dropout_rate': [0.1, 0.2, 0.3],
    'learning_rate': [0.001, 0.0001],
    'batch_size': [16, 32]
}

# Grid search with early stopping
results = []

print("Starting Grid Search with Early Stopping...")
print("=" * 70)

total_combinations = (len(hyperparameters['units']) * 
                     len(hyperparameters['dropout_rate']) * 
                     len(hyperparameters['learning_rate']) * 
                     len(hyperparameters['batch_size']))

iteration = 0

for units in hyperparameters['units']:
    for dropout in hyperparameters['dropout_rate']:
        for lr in hyperparameters['learning_rate']:
            for batch_size in hyperparameters['batch_size']:
                
                iteration += 1
                print(f"\n[{iteration}/{total_combinations}] Testing configuration:")
                print(f"  units={units}, dropout={dropout}, lr={lr}, batch_size={batch_size}")
                
                model, val_loss, epochs_trained, history = build_and_train_lstm_2(
                    units, dropout, lr, batch_size, max_epochs=200
                )
                
                results.append({
                    'units': units,
                    'dropout': dropout,
                    'learning_rate': lr,
                    'batch_size': batch_size,
                    'val_loss': val_loss,
                    'epochs_trained': epochs_trained
                })
                
                print(f"  Best Val Loss: {val_loss:.6f}")
                print(f"  Epochs Trained: {epochs_trained} (stopped early)")

print("\n" + "=" * 70)
print("Grid Search Complete!")

# Convert to DataFrame and analyze
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('val_loss')

print("\n📊 Top 5 Configurations:")
print(results_df.head(10))

print("\n🏆 Best Configuration:")
best = results_df.iloc[0]
print(f"  Units: {best['units']}")
print(f"  Dropout: {best['dropout']}")
print(f"  Learning Rate: {best['learning_rate']}")
print(f"  Batch Size: {best['batch_size']}")
print(f"  Best Val Loss: {best['val_loss']:.6f}")
print(f"  Epochs: {best['epochs_trained']}")

# Save results
results_df.to_csv('grid_search_results.csv', index=False)
print("\n✅ Results saved to 'grid_search_results.csv'")

Starting Grid Search with Early Stopping...

[1/36] Testing configuration:
  units=32, dropout=0.1, lr=0.001, batch_size=16
  Best Val Loss: 0.011538
  Epochs Trained: 200 (stopped early)

[2/36] Testing configuration:
  units=32, dropout=0.1, lr=0.001, batch_size=32
  Best Val Loss: 0.024174
  Epochs Trained: 19 (stopped early)

[3/36] Testing configuration:
  units=32, dropout=0.1, lr=0.0001, batch_size=16
  Best Val Loss: 0.024117
  Epochs Trained: 32 (stopped early)

[4/36] Testing configuration:
  units=32, dropout=0.1, lr=0.0001, batch_size=32
  Best Val Loss: 0.024018
  Epochs Trained: 39 (stopped early)

[5/36] Testing configuration:
  units=32, dropout=0.2, lr=0.001, batch_size=16
  Best Val Loss: 0.014281
  Epochs Trained: 200 (stopped early)

[6/36] Testing configuration:
  units=32, dropout=0.2, lr=0.001, batch_size=32
  Best Val Loss: 0.021758
  Epochs Trained: 200 (stopped early)

[7/36] Testing configuration:
  units=32, dropout=0.2, lr=0.0001, batch_size=16
  Best Val L